## This is very important!
You need to generate the dataset yourself or download the existing one! You can do dynamic generation on the go, but it usually takes around 1.5 hours to go through the dataset. I prefer to use statically generated dataset because it allows to iterate throught the dataset in am matter of 10 minutes. Frame size in the number of actions you want to be in the state (the number of movies rated). I have chosen it to be 10, if you want another number you will have to generate it yourself. Link to the google drive is in the readme.md

In [ ]:
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import json
import pickle

cuda = torch.device('cpu')
frame_size = 10
ratings = pd.read_csv('../data/ml-20m/ratings_lite.csv')
movies = pickle.load(open('../data/infos_pca128.pytorch', 'rb'))
infos_web = json.load(open('../data/infos.json')) 

# credits: KnightofK9
ratings["rating"] = ratings["rating"].apply(lambda i: 2 * (i - 2.5))
users = ratings[["userId","movieId"]].groupby(["userId"]).size()
users = users[users >= frame_size + 1]

In [ ]:
test_users = users[-100:]
train_users = users[:-100]

train_ratings = ratings[ratings["userId"].isin(train_users.index)]
test_ratings = ratings[ratings["userId"].isin(test_users.index)]

train_ratings = train_ratings.sort_values(by=["userId", "timestamp"]).drop(columns=["timestamp"]).set_index("userId")
test_ratings = test_ratings.sort_values(by=["userId", "timestamp"]).drop(columns=["timestamp"]).set_index("userId")

for i in movies.keys():
    movies[i] = movies[i].to(cuda)

In [ ]:
import io
import matplotlib.pyplot as plt
import pickle

n_iter = 1
n_batch = 1
n_user = 0


def get_minibatch(df, idx):
    user_ratings = df[idx:frame_size + idx + 1]
    user_ratings = user_ratings[["movieId", "rating"]].values

    chosen_movie = user_ratings[:, 0][-1] 
    chosen_movie = movies[chosen_movie] # action
    chosen_rating = user_ratings[:, 1][-1] # reward
    films_watched = user_ratings[:, 0][:-1]
    watched_rating = user_ratings[:, 1][:-1] # state
    watched_infos = [movies[i] for i in films_watched] # state
    watched_infos = torch.stack(watched_infos)
    next_infos = torch.cat((watched_infos[1:], chosen_movie.unsqueeze(0)), 0)
    next_rating = watched_rating[1:].tolist()
    next_rating.append(chosen_rating)
    next_rating = torch.tensor(next_rating)

    # state action reward next_state done
    return [(watched_infos, watched_rating), chosen_movie, chosen_rating,
                  (next_infos, next_rating), idx + 1 == size]


batch_bar = tqdm(total=len(train_users))
batch = []
batch_size = 100

for user, df in train_ratings.groupby(level=0):
    batch_bar.update(1)
    n_user += 1
    size = max(len(df) - frame_size, 0)
    for idx in range(0, size):
        if np.random.rand() < 0.8:  # intake percents
            continue
        batch.append(get_minibatch(df, idx))
    
    # if the memory error pops up, change the number below
    # 8gb ram = 5000
    # 16gb ram = 10000
    # 32gb ram = 30000
    if n_user % 10000 == 0:
        pickle.dump(batch, open('../data/batches/batch_{}.p'.format(n_batch), 'wb'))
        n_batch += 1
        del batch
        batch = []


In [ ]:
pickle.dump(batch, open('../data/batches/batch_{}.p'.format(n_batch), 'wb'))
n_batch += 1
del batch
batch = []

## H5PY integration
p.s. restart the kernel

In [ ]:
import h5py
import numpy as np
import torch
import pickle

h5 = h5py.File("*path to where ypu want to save it*", "w")
idx = 0
# datasets:
# ds_state (float) (None, (frame_size + 1) * embed_size): (None, 1290)
# ds_action (float) (None, embed_size): (None, 128)
# ds_reward (int4) (None)
# ds_next_state (float) (None, (frame_size + 1) * embed_size): (None, 1290)
# ds_done (bool) (None)

In [ ]:
ds_state = h5.create_dataset("state", (0, 1290), maxshape=(None, 1290), dtype='f', chunks=True, compression="lzf")
ds_action = h5.create_dataset("action", (0, 128), maxshape=(None, 128), dtype='f', chunks=True,  compression="lzf")
ds_reward = h5.create_dataset("reward", (0, ), maxshape=(None,), dtype='i1',  chunks=True,  compression="lzf")
ds_next_state = h5.create_dataset("next_state", (0, 1290), maxshape=(None, 1290), dtype='f',  chunks=True, compression="lzf")
ds_done = h5.create_dataset("done", (0, ), maxshape=(None,), dtype='?', chunks=True,  compression="lzf")

In [ ]:
from tqdm import tqdm_notebook as tqdm

# frame size you've chosen
# must match with the number in the upper snippet
frame_size = 10

for j in tqdm(range(1,15)):
    f = pickle.load(open('../data/batches/batch_{}.p'.format(j), 'rb'))
    [i.resize([len(f) + i.shape[0], i.shape[1]]) for i in [ds_state,
                                                ds_action, ds_next_state ]]
    [i.resize([len(f) + i.shape[0]]) for i in [ds_reward, ds_done]]


    for i in tqdm(f):
        infos, ratings = i[0]
        # frame_size, embed_size -> frame_size * embed_size
        infos = infos.view(-1).numpy()
        state = np.concatenate([infos, ratings])

        action = i[1].numpy()
        reward = i[2]

        next_infos, next_ratings = i[3]
        next_infos = next_infos.view(-1).numpy()
        next_state = np.concatenate([next_infos, next_ratings])
        done = i[4]

        ds_state[idx] = state.astype('float16')
        ds_action[idx] = action.astype('float16')
        ds_reward[idx] = reward.astype('int8')
        ds_next_state[idx] = next_state.astype('float16')
        ds_done[idx] = done

        idx+=1

## Test:

In [ ]:
import torch
import pickle
from tqdm import tqdm_notebook as tqdm
import h5py
import numpy as np

f = h5py.File("*your path here*", "r")
#/media/dev/New Volume/projects/RecNN/static_dataset.hdf5
#f = h5

cuda = torch.device('cuda')

def prepare_batch(*args):
    args = [torch.tensor(np.array(arg).astype(np.float)).to(cuda) for arg in args]
    return args


batch = []
batch_size = 5000
losses = []
T_losses = []


for i in tqdm(range(f['state'].shape[0] // batch_size)):
    batch = [f[key][i*batch_size:(i+1)*batch_size] for key in
             ['state', 'action', 'reward', 'next_state', 'done']]
    
    batch = prepare_batch(*batch)
    batch = []
